# Graph Neural Networks Implementation
This notebook implements different types of Graph Neural Networks using PyTorch Geometric (PyG)

In [1]:
# Install required packages if not already installed
!pip install torch-geometric

In [3]:
# Import required libraries
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, SAGEConv, GATConv

## Load and Explore Dataset

In [4]:
# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Get the graph data
data = dataset[0]

# Print dataset statistics
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Number of features: {data.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Processing...


Number of nodes: 2708
Number of edges: 10556
Number of features: 1433
Number of classes: 7


Done!


## Graph Convolutional Network (GCN)

In [5]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

## GraphSAGE Implementation

In [6]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

## Graph Attention Network (GAT)

In [7]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=8, dropout=0.6):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, dropout=dropout)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

## Training and Evaluation Function

In [8]:
def train_and_evaluate(model, data, epochs=200):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = torch.nn.NLLLoss()
    
    # Training
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 50 == 0:
            print(f'Epoch {epoch+1:03d}, Loss: {loss.item():.4f}')
    
    # Evaluation
    model.eval()
    _, pred = model(data.x, data.edge_index).max(dim=1)
    correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    accuracy = correct / data.test_mask.sum().item()
    print(f'Test Accuracy: {accuracy:.4f}')
    
    return accuracy

## Train and Compare Models

In [9]:
# Model parameters
in_channels = dataset.num_node_features
hidden_channels = 16
out_channels = dataset.num_classes

# Initialize models
gcn_model = GCN(in_channels, hidden_channels, out_channels)
sage_model = GraphSAGE(in_channels, hidden_channels, out_channels)
gat_model = GAT(in_channels, hidden_channels, out_channels)

# Train and evaluate each model
print("Training GCN...")
gcn_acc = train_and_evaluate(gcn_model, data)

print("\nTraining GraphSAGE...")
sage_acc = train_and_evaluate(sage_model, data)

print("\nTraining GAT...")
gat_acc = train_and_evaluate(gat_model, data)

# Compare results
print("\nFinal Results:")
print(f"GCN Accuracy: {gcn_acc:.4f}")
print(f"GraphSAGE Accuracy: {sage_acc:.4f}")
print(f"GAT Accuracy: {gat_acc:.4f}")

Training GCN...
Epoch 050, Loss: 0.0619
Epoch 100, Loss: 0.0328
Epoch 150, Loss: 0.0300
Epoch 200, Loss: 0.0346
Test Accuracy: 0.7950

Training GraphSAGE...
Epoch 050, Loss: 0.0217
Epoch 100, Loss: 0.0266
Epoch 150, Loss: 0.0175
Epoch 200, Loss: 0.0109
Test Accuracy: 0.7830

Training GAT...
Epoch 050, Loss: 0.3255
Epoch 100, Loss: 0.3336
Epoch 150, Loss: 0.3269
Epoch 200, Loss: 0.3211
Test Accuracy: 0.7900

Final Results:
GCN Accuracy: 0.7950
GraphSAGE Accuracy: 0.7830
GAT Accuracy: 0.7900
